In [1]:
import numpy as np
import itertools
import sys
import CsemigroupsCPP as CPP

### Auxiliar functions

In [15]:
def ComputeCone(bound,eq):
    '''
    Compute the points of a cone until a bound.
    '''
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq):
            cone.append(list(x))
    return cone

def PrecGradLex(a,b):
    '''
    Check if A<B
    '''
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        for x in c:
            if x > 0:
                return True
            if x < 0:
             return False
        
def PrecGradRevLex(a,b):
    '''
    Check if A<B
    '''
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        c = list(c)
        c.reverse()
        for x in c:
            if x < 0:
                return True
            if x > 0:
             return False
def PrecTest(a,b):
    '''
    Check if A<B
    '''
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        c = list(c)
        c.reverse()
        for x in c:
            if x > 0:
                return True
            if x < 0:
             return False
        
def MaximumOrder(v,order):
    '''
    Return the maximum element of v according to an order
    '''
    maximum = v[0]
    for x in v:
        if order(maximum,x):
            maximum = x
    return maximum

def MinimumOrder(v,order):
    '''
    Return the minimum element of v according to an order
    '''
    minimum = v[0]
    for x in v:
        if order(x,minimum):
            minimum = x
    return minimum

def GetSemigroupFromGaps(gaps,eq,order):
    '''
    Compute a C-semigroup from a gapset
    '''    
    frob = list(MaximumOrder(gaps,order))

    mfrob = max(frob)
    for i in range(len(frob)):
        if frob[i] == 0:
            frob[i] = mfrob
    
    bound =  [3*frob[i] for i in range(len(frob))]
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq) and not CPP.allZero(list(x)):
            cone.append(list(x))
    print("Cone = ",cone)
    print("gaps =",gaps)
    for x in gaps:
        cone.remove(x)
    cs = CPP.Csemigroup(cone)
    return cs

### Trees of irreducible $C$-semigroups

In [5]:
def ComputeIC(cone,frob,eq):
    aux = []
    for x in cone:
        if CPP.belongCone([frob[i]-x[i] for i in range(len(x))],eq):
            aux.append(x)
    return aux

def ComputeRootTreeIrreductible(IC, frob, order,eq):
    hFrob = [x/2 for x in frob]
    i1 = []
    for x in IC:
        if not CPP.allZero(list(x)) and not order(hFrob,x):
            i1.append(x)
    i1.append(frob)
    return GetSemigroupFromGaps(i1,eq,order)

def GetGeneratorsBetweenHalfFrobAndFrob(gen, frob,order):
    hFrob = [x/2 for x in frob]
    i2 = []
    for x in gen:
        aux = list(np.array(frob)-np.array(x))
        if order(hFrob,x) and CPP.allPositive([frob[i]-x[i] for i in range(len(x))]):
            i2.append(x)
    return i2

def CheckPropertiesGeneratorsTree(gen, frob,IC,order):
    aux = []
    candidates = GetGeneratorsBetweenHalfFrobAndFrob(gen, frob,order)
    for x in candidates:
        if x in IC and not CPP.belongByGens([2*x[i]-frob[i] for i in range(len(x))],gen) and [3*x[i] for i in range(len(x))] != [2*frob[i] for i in range(len(x))] and [4*x[i] for i in range(len(x))] != [3*frob[i] for i in range(len(x))] and order([frob[i]-x[i] for i in range(len(x))],MinimumOrder(gen,order)):
            aux.append(x)
    return aux

def ComputeIrreducibleChildren(sg,frob,order, ic,eq):
    children = []
    A = CheckPropertiesGeneratorsTree(sg.GetGenerators(),frob,ic,order)
    print("A =",A)
    for x in A:
        gaps = list(sg.GetGaps())
        gaps.append(x)
        gaps.remove([frob[i]-x[i] for i in range(len(x))])
        children.append(GetSemigroupFromGaps(gaps,eq,order))
    return children

def ComputeTreeIrreducible(frob, eq, order):
    bound =  [3*frob[i] for i in range(len(frob))]### <--- COMPROBAR SI ESTA COTA SE PUEDE MEJORAR!!!
    cone = ComputeCone(bound,eq)
    ic = ComputeIC(cone,frob,eq)
    sf = ComputeRootTreeIrreductible(ic, frob, order,eq)
    print("Los generadores del padre son:", sf.GetGenerators())
    X = [sf]
    Y = []
    while X != []:
        print(len(X))
        children = []
        for sg in X:
            print("Padre:",sg.GetGenerators())
            print("Hijo:")
            children = children + ComputeIrreducibleChildren(sg,frob,order, ic,eq)
            for kk in children:
                print(kk.GetGenerators())
            Y.append([sg,children])
        X = children

        print("\n")
    return(Y)

#### Example I

In [6]:
cone = CPP.Csemigroup([[1,0],[1,1],[1,2]])
cone.GetHyperplanes()
eq = [[0, 1], [2, -1]]
frob = [4,2]
cone = ComputeCone([24,18],eq)
ic = ComputeIC(cone,frob,eq)
sf = ComputeRootTreeIrreductible(ic, frob, PrecGradLex,eq)
aa = sf.GetGenerators()
arbol  = ComputeTreeIrreducible([4,2], [[0, 1], [2, -1]], PrecGradLex)

Los generadores del padre son: [[3, 0], [4, 0], [5, 0], [3, 1], [4, 1], [5, 1], [2, 2], [3, 2], [2, 3], [3, 3], [4, 3], [2, 4], [3, 4], [3, 5], [3, 6]]
1
Padre: [[3, 0], [4, 0], [5, 0], [3, 1], [4, 1], [5, 1], [2, 2], [3, 2], [2, 3], [3, 3], [4, 3], [2, 4], [3, 4], [3, 5], [3, 6]]
Hijo:
A = [[3, 0], [3, 1], [2, 2]]
[[4, 0], [5, 0], [6, 0], [7, 0], [3, 1], [4, 1], [5, 1], [6, 1], [1, 2], [2, 2], [3, 2], [2, 3], [3, 3]]
[[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [2, 3], [2, 4], [3, 6]]
[[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [2, 3], [3, 3], [2, 4], [3, 4], [3, 5], [4, 5], [3, 6]]


3
Padre: [[4, 0], [5, 0], [6, 0], [7, 0], [3, 1], [4, 1], [5, 1], [6, 1], [1, 2], [2, 2], [3, 2], [2, 3], [3, 3]]
Hijo:
A = [[3, 1], [2, 2]]
[[4, 0], [5, 0], [6, 0], [7, 0], [1, 1], [4, 1], [1, 2], [3, 2]]
[[2, 0], [5, 0], [3, 1], [4, 1], [1, 2], [2, 3], [3, 3], [3, 4]]
Padre: [[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [2, 3], [2, 4], [3, 6]]
Hijo:
A = []
[[4, 0], [5, 0], [6, 0], [7, 0], [1, 1], [4, 1], [1, 2], [

#### Example II

In [7]:
cone = CPP.Csemigroup([[1,0],[1,1],[1,2]])
cone.GetHyperplanes()
eq = [[0, 1], [2, -1]]
frob = [4,2]
cone = ComputeCone([24,18],eq)
ic = ComputeIC(cone,frob,eq)
sf = ComputeRootTreeIrreductible(ic, frob, PrecGradLex,eq)
aa = sf.GetGenerators()
arbol2  = ComputeTreeIrreducible([4,2], [[0, 1], [2, -1]], PrecTest)

Los generadores del padre son: [[4, 0], [5, 0], [6, 0], [7, 0], [3, 1], [4, 1], [5, 1], [6, 1], [1, 2], [2, 2], [3, 2], [2, 3], [3, 3]]
1
Padre: [[4, 0], [5, 0], [6, 0], [7, 0], [3, 1], [4, 1], [5, 1], [6, 1], [1, 2], [2, 2], [3, 2], [2, 3], [3, 3]]
Hijo:
A = [[3, 1], [1, 2], [2, 2]]
[[4, 0], [5, 0], [6, 0], [7, 0], [1, 1], [4, 1], [1, 2], [3, 2]]
[[3, 0], [4, 0], [5, 0], [3, 1], [4, 1], [5, 1], [2, 2], [3, 2], [2, 3], [3, 3], [4, 3], [2, 4], [3, 4], [3, 5], [3, 6]]
[[2, 0], [5, 0], [3, 1], [4, 1], [1, 2], [2, 3], [3, 3], [3, 4]]


3
Padre: [[4, 0], [5, 0], [6, 0], [7, 0], [1, 1], [4, 1], [1, 2], [3, 2]]
Hijo:
A = []
Padre: [[3, 0], [4, 0], [5, 0], [3, 1], [4, 1], [5, 1], [2, 2], [3, 2], [2, 3], [3, 3], [4, 3], [2, 4], [3, 4], [3, 5], [3, 6]]
Hijo:
A = [[3, 1], [2, 2]]
[[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [2, 3], [2, 4], [3, 6]]
[[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [2, 3], [3, 3], [2, 4], [3, 4], [3, 5], [4, 5], [3, 6]]
Padre: [[2, 0], [5, 0], [3, 1], [4, 1], [1, 2], [2, 3], [3

### Fundamental gaps

In [10]:
def DeleteDuplicates(v):
    w = []
    for x in v:
        if x not in w:
            w.append(x)
    return w

def Subset(v,w):
    for x in v:
        if x not in w:
            return False
    return True

def ComputeMaximimumPerCoordinate(v):
    n = len(v[0])
    maximum  = []
    for i in range(n):
        aux = 0
        for x in v:
            if x[i] > aux:
                aux = x[i]
        maximum.append(aux)
    return maximum

def ComputeD(eqCone, X):
    bound = ComputeMaximimumPerCoordinate(X)
    points = ComputeCone(bound, eqCone)
    points = [x for x in points if not CPP.allZero(x)]
    candidates = list(points)
    D = []
    maxN = max(bound)
    for i in range(1,maxN+1):
        candidates = [[i*x[j] for j in range(len(x))] for x in points]
        #print("i = ",i,"candidates = ",candidates)
        for j in range(len(candidates)):
            if candidates[j] in X:
                D.append(points[j])
    #print(points)
    #print(X)
    return D

def ConeMinusSet(cone, X, order):
    genCone = cone.GetGenerators()
    eqCone = cone.GetHyperplanes()
    if Subset(X,genCone):
        return GetSemigroupFromGaps(X,eqCone,order)
    D = DeleteDuplicates(ComputeD(eqCone,X))
    print("D=",D)
    x1 = MinimumOrder(X,order)
    if not Subset(X,D) or not Subset(D,X):
        raise Exception("C\X is not a C-semigroup") 
    gen = [x for x in genCone]
    t = len(X)
    gaps = []
    aux = X
    for i in range(t):
        print("\n t=",i)
        print("Estudiamos ",X[i])
        if X[i] not in gen:
            raise Exception("C\X is not a C-semigroup")
        gaps.append(X[i])
        print("gaps = ",gaps)
        gen = GetSemigroupFromGaps(gaps,eqCone,order).GetGenerators()
        print("Gen =",gen)
        aux = [x for x in aux if x != X[i]]
        if Subset(aux,gen):
            return GetSemigroupFromGaps(gaps+aux,eqCone,order)

#### Example I

In [11]:
gen = [[1,0],[1,1],[1,2]]
X1 = [[1,0], [1,1]]
cone = CPP.Csemigroup(gen)
ConeMinusSet(cone, X1, PrecGradLex).GetGenerators()

[[2, 0], [3, 0], [2, 1], [3, 1], [1, 2], [2, 2], [2, 3]]

#### Example II

In [12]:
X2 = [[2,0],[2,1]]
cone = CPP.Csemigroup(gen)
ConeMinusSet(cone, X2, PrecGradLex).GetGenerators()

D= [[2, 0], [2, 1], [1, 0]]


Exception: C\X is not a C-semigroup

#### Example III

In [16]:
gen = [[1,0],[1,1],[1,2]]
X3 = [[1,0],[1,1],[1,2],[2,0],[2,1],[2,2],[2,3],[2,4]]
cone = CPP.Csemigroup(gen)
ConeMinusSet(cone, X3, PrecGradLex).GetGenerators()

D= [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4]]

 t= 0
Estudiamos  [1, 0]
gaps =  [[1, 0]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2], [3, 3]]
gaps = [[1, 0]]
Gen = [[2, 0], [3, 0], [1, 1], [2, 1], [1, 2]]

 t= 1
Estudiamos  [1, 1]
gaps =  [[1, 0], [1, 1]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2], [3, 3]]
gaps = [[1, 0], [1, 1]]
Gen = [[2, 0], [3, 0], [2, 1], [3, 1], [1, 2], [2, 2], [2, 3]]

 t= 2
Estudiamos  [1, 2]
gaps =  [[1, 0], [1, 1], [1, 2]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6]]
gaps = [[1, 0], [1, 1], [1, 2]]
Gen = [[2, 0], [3, 0], [2, 1], [3, 1], [2, 2], [3, 2], [2, 3], [3, 3], [2, 4], [3, 4], [3, 5], [3, 6]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4

[[3, 0],
 [4, 0],
 [5, 0],
 [3, 1],
 [4, 1],
 [5, 1],
 [3, 2],
 [4, 2],
 [5, 2],
 [3, 3],
 [4, 3],
 [5, 3],
 [3, 4],
 [4, 4],
 [5, 4],
 [3, 5],
 [4, 5],
 [5, 5],
 [3, 6],
 [4, 6],
 [5, 6],
 [4, 7],
 [5, 7],
 [4, 8],
 [5, 8],
 [5, 9],
 [5, 10]]